In [1]:
import pandas as pd
import plotly.express as px

## Задание 1

Найти 10 самых крупных аэропортов в Мире по сумме взлет/посадок. Результат в виде DataFrame, индекс 0 уровня Страна, индекс 1 уровня Название аэропорта, колонки взлеты, посадки, суммы взлет/посадок. Важно упорядочить данные от наибольшего к наименьшему.

In [2]:
routes_columns = [
    "Airline", "Airline ID", "Source airport", "Source airport ID",
    "Destination airport", "Destination airport ID",
    "Codeshare", "Stops", "Equipment"
]
airports_columns = [
    "Airport ID", "Name", "City", "Country", "IATA", "ICAO",
    "Latitude", "Longitude", "Altitude", "Timezone", "DST",
    "Tz database timezone", "Type", "Source"
]
routes = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat', header=None, names=routes_columns)
airports = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', header=None, names=airports_columns)


In [3]:
takeoff = routes['Source airport'].value_counts().rename('Takeoff')
landings = routes['Destination airport'].value_counts().rename('Landings')
flights = pd.concat([takeoff,landings],axis = 1).fillna(0)
flights['Total'] = flights['Takeoff'] + flights['Landings']
flights = flights.reset_index().rename(columns={'index': 'IATA'})
airports_with_flights = airports.merge(flights, left_on = 'IATA' , right_on = 'IATA',how = 'inner')
airports_with_flights = airports_with_flights.set_index(['Country', 'Name'])
result = airports_with_flights[['Takeoff', 'Landings', 'Total']]
result = result.sort_values('Total', ascending=False)
top_10_airports = result.head(10)
top_10_airports

Takeoff  \
Country        Name                                                        
United States  Hartsfield Jackson Atlanta International Airport    915.0   
               Chicago O'Hare International Airport                558.0   
China          Beijing Capital International Airport               535.0   
United Kingdom London Heathrow Airport                             527.0   
France         Charles de Gaulle International Airport             524.0   
United States  Los Angeles International Airport                   492.0   
Germany        Frankfurt am Main Airport                           497.0   
United States  Dallas Fort Worth International Airport             469.0   
               John F Kennedy International Airport                456.0   
Netherlands    Amsterdam Airport Schiphol                          453.0   

                                                                 Landings  \
Country        Name                                                         
United States  Hartsfield Jackson Atlanta International Airport     911.0   
               Chicago O'Hare International Airport                 550.0   
China          Beijing Capital International Airport                534.0   
United Kingdom London Heathrow Airport                              524.0   
France         Charles de Gaulle International Airport              517.0   
United States  Los Angeles International Airport                    498.0   
Germany        Frankfurt am Main Airport                            493.0   
United States  Dallas Fort Worth International Airport              467.0   
               John F Kennedy International Airport                 455.0   
Netherlands    Amsterdam Airport Schiphol                           450.0   

                                                                  Total  
Country        Name                                                      
United States  Hartsfield Jackson Atlanta International Airport  1826.0  
               Chicago O'Hare International Airport              1108.0  
China          Beijing Capital International Airport             1069.0  
United Kingdom London Heathrow Airport                           1051.0  
France         Charles de Gaulle International Airport           1041.0  
United States  Los Angeles International Airport                  990.0  
Germany        Frankfurt am Main Airport                          990.0  
United States  Dallas Fort Worth International Airport            936.0  
               John F Kennedy International Airport               911.0  
Netherlands    Amsterdam Airport Schiphol                         903.0

## Задание 2

Требуется найти проблемные данные, ошибочные данные, несоответствия в данных и т.п. Результаты надо сгруппировать по типам выявленных ошибок.

### Ошибки в аэропортах

#### 1. Пропуски + ограничения на длину

In [12]:
airports[(airports['IATA'].isnull()) | (airports['IATA'] == '\\N') | (airports['IATA'].str.len() != 3)]

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database timezone,Type,Source
21,22,Winnipeg / St. Andrews Airport,Winnipeg,Canada,\N,CYAV,50.056400,-97.032501,760,-6,A,America/Winnipeg,airport,OurAirports
22,23,Halifax / CFB Shearwater Heliport,Halifax,Canada,\N,CYAW,44.639702,-63.499401,144,-4,A,America/Halifax,airport,OurAirports
43,44,Princeton Airport,Princeton,Canada,\N,CYDC,49.468102,-120.511002,2298,-8,A,America/Vancouver,airport,OurAirports
103,104,Pitt Meadows Airport,Pitt Meadows,Canada,\N,CYPK,49.216099,-122.709999,11,-8,A,America/Vancouver,airport,OurAirports
210,212,Boufarik Airport,Boufarik,Algeria,\N,DAAK,36.545799,2.876110,335,1,N,Africa/Algiers,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7692,14105,Kubinka Air Base,Kubinka,Russia,\N,UUMB,55.611695,36.650002,614,\N,\N,\N,airport,OurAirports
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298,272,\N,\N,\N,airport,OurAirports
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports


In [8]:
airports[(airports['ICAO'].isnull()) | (airports['ICAO'] == '\\N') | (airports['ICAO'].str.len() != 4)]

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database timezone,Type,Source
4435,5743,Kili Airport,Kili Island,Marshall Islands,KIO,Q51,5.644515,169.119507,5,12,U,Pacific/Majuro,airport,OurAirports
4501,5814,Utirik Airport,Utirik Island,Marshall Islands,UTK,03N,11.222000,169.852005,4,12,U,Pacific/Majuro,airport,OurAirports
4765,6126,Aleknagik / New Airport,Aleknagik,United States,WKK,5A8,59.282600,-158.617996,66,-9,A,America/Anchorage,airport,OurAirports
4769,6134,Akutan Seaplane Base,Akutan,United States,KQA,KQA,54.133770,-165.778896,0,-9,A,America/Anchorage,airport,OurAirports
4770,6136,Lopez Island Airport,Lopez,United States,LPS,S31,48.483898,-122.938004,209,-8,A,America/Los_Angeles,airport,OurAirports
5860,7909,Sun Island Resort and SPA,South Aari Atoll,Maldives,\N,\N,3.488334,72.862989,0,-3,N,America/Sao_Paulo,airport,OurAirports


In [9]:
airports[(airports['Timezone'].isnull()) | (airports['Timezone'] == '\\N')]


,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database timezone,Type,Source
6981,11743,La Grande-4 Airport,La Grande-4,Canada,YAH,CYAH,53.754700,-73.675301,1005,\N,\N,\N,airport,OurAirports
6982,11744,Alert Bay Airport,Alert Bay,Canada,YAL,CYAL,50.582199,-126.916000,240,\N,\N,\N,airport,OurAirports
6983,11745,Liverpool South Shore Regional Airport,Liverpool,Canada,\N,CYAU,44.230301,-64.856102,321,\N,\N,\N,airport,OurAirports
6984,11747,Nipawin Airport,Nipawin,Canada,\N,CYBU,53.332500,-104.008003,1220,\N,\N,\N,airport,OurAirports
6985,11748,Centralia / James T. Field Memorial Aerodrome,Centralia,Canada,YCE,CYCE,43.285599,-81.508301,824,\N,\N,\N,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298,272,\N,\N,\N,airport,OurAirports
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports


In [10]:
airports[(airports['DST'].isnull()) | (airports['DST'] == '\\N') | (airports['DST'].str.len() != 1)]

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database timezone,Type,Source
6981,11743,La Grande-4 Airport,La Grande-4,Canada,YAH,CYAH,53.754700,-73.675301,1005,\N,\N,\N,airport,OurAirports
6982,11744,Alert Bay Airport,Alert Bay,Canada,YAL,CYAL,50.582199,-126.916000,240,\N,\N,\N,airport,OurAirports
6983,11745,Liverpool South Shore Regional Airport,Liverpool,Canada,\N,CYAU,44.230301,-64.856102,321,\N,\N,\N,airport,OurAirports
6984,11747,Nipawin Airport,Nipawin,Canada,\N,CYBU,53.332500,-104.008003,1220,\N,\N,\N,airport,OurAirports
6985,11748,Centralia / James T. Field Memorial Aerodrome,Centralia,Canada,YCE,CYCE,43.285599,-81.508301,824,\N,\N,\N,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298,272,\N,\N,\N,airport,OurAirports
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports


In [22]:
airports[airports['City'].isna()]

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database timezone,Type,Source
7031,11794,Minsk Mazowiecki Military Air Base,NaN,Poland,\N,EPMM,52.195499,21.655899,604,\N,\N,\N,airport,OurAirports
7032,11795,Powidz Military Air Base,NaN,Poland,\N,EPPW,52.379398,17.853901,371,\N,\N,\N,airport,OurAirports
7137,11900,King Salman Abdulaziz Airport,NaN,Saudi Arabia,DWD,OEDM,24.449900,44.121201,3026,\N,\N,\N,airport,OurAirports
7138,11901,King Khaled Air Base,NaN,Saudi Arabia,KMX,OEKM,18.297300,42.803501,6778,\N,\N,\N,airport,OurAirports
7158,11921,Asahikawa Airfield,NaN,Japan,\N,RJCA,43.794734,142.365432,377,\N,\N,\N,airport,OurAirports
7160,11923,Utsunomiya Airport,NaN,Japan,\N,RJTU,36.514500,139.871010,334,\N,\N,\N,airport,OurAirports
7161,11924,Jungwon Air Base/Chungju Airport,NaN,South Korea,\N,RKTI,37.030240,127.886353,281,\N,\N,\N,airport,OurAirports
7164,11927,Bislig Airport,NaN,Philippines,BPH,RPMF,8.195950,126.321999,12,\N,\N,\N,airport,OurAirports
7165,11928,Mati National Airport,NaN,Philippines,MXI,RPMQ,6.949091,126.273680,156,\N,\N,\N,airport,OurAirports
7184,11947,Metropolitano Airport,NaN,Venezuela,\N,SVMP,10.133169,-66.787827,574,\N,\N,\N,airport,OurAirports


In [11]:
airports[(airports['Tz database timezone'].isnull()) | (airports['Tz database timezone'] == '\\N')]

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database timezone,Type,Source
1846,1907,Leeward Point Field,Guantanamo,Cuba,NBW,MUGM,19.906500,-75.207100,56,-5,U,\N,airport,OurAirports
1899,1966,Tabiteuea North Airport,Tabiteuea North,Kiribati,TBF,NGTE,-1.224470,174.776001,7,12,U,\N,airport,OurAirports
1903,1971,Rurutu Airport,Rurutu,French Polynesia,RUR,NTAR,-22.434099,-151.360992,18,-10,U,\N,airport,OurAirports
1908,1976,Reao Airport,Reao,French Polynesia,REA,NTGE,-18.465900,-136.440002,12,-10,U,\N,airport,OurAirports
1924,1993,Hao Airport,Hao Island,French Polynesia,HOI,NTTO,-18.074800,-140.945999,10,-10,U,\N,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298,272,\N,\N,\N,airport,OurAirports
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports


#### 2. Неполные данные

In [14]:
tmp = (airports[['IATA', 'Country', 'Name']]
       .merge(flights, on='IATA', how='right')
       .set_index(['Country', 'Name']))

In [15]:
tmp['IATA'].isin(airports.IATA.unique())

Country         Name                                            
United States   Hartsfield Jackson Atlanta International Airport     True
                Chicago O'Hare International Airport                 True
China           Beijing Capital International Airport                True
United Kingdom  London Heathrow Airport                              True
France          Charles de Gaulle International Airport              True
                                                                    ...  
Mongolia        Dalanzadgad Airport                                  True
Honduras        Utila Airport                                        True
NaN             NaN                                                 False
Ecuador         Teniente Coronel Luis a Mantilla Airport             True
Greece          Filippos Airport                                     True
Name: IATA, Length: 3425, dtype: bool

**В аэропортах не все данные, что в маршрутах**

In [17]:
tmp[~tmp['IATA'].isin(airports.IATA.unique())]

IATA  Takeoff  Landings  Total
Country Name                               
NaN     NaN   INC     65.0      66.0  131.0
        NaN   NTG     18.0      18.0   36.0
        NaN   MRA     12.0      12.0   24.0
        NaN   MLH     11.0      12.0   23.0
        NaN   FSZ     10.0      10.0   20.0
...           ...      ...       ...    ...
        NaN   KPR      0.0       1.0    1.0
        NaN   CZJ      0.0       1.0    1.0
        NaN   KZB      0.0       1.0    1.0
        NaN   MTE      0.0       1.0    1.0
        NaN   QFX      0.0       1.0    1.0

[163 rows x 4 columns]

#### 3. Проблемы в отображении страны или несоответсвие с шириной и долготой

In [21]:
airports[airports["Name"] == "Hunt Field"]

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database timezone,Type,Source
5814,7831,Hunt Field,Lindau,Germany,LND,KLND,42.815201,-108.730003,5586,-7,E,America/Denver,airport,OurAirports


**Пишется, что находится в Германии, а на самом деле в США**

### Ошибки в маршрутах

In [23]:
routes[routes.Codeshare.isna()]

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [24]:
routes[routes['Source airport ID'] == '\\N']

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
38,2B,410,TGK,\N,DME,4029,NaN,0,CR2
54,2G,1654,KCK,\N,IKT,2937,NaN,0,AN4
1163,4O,17885,PQM,\N,MEX,1824,NaN,0,SU9
2113,5Z,18946,PBZ,\N,CPT,797,NaN,0,BEH
2114,5Z,18946,PBZ,\N,JNB,813,NaN,0,BEH
...,...,...,...,...,...,...,...,...,...
66353,Z8,603,ORU,\N,SRZ,4247,NaN,0,SWM
67468,ZI,21,MLH,\N,ALG,210,NaN,0,320
67469,ZI,21,MLH,\N,CZL,221,NaN,0,320
67470,ZI,21,MLH,\N,ORN,231,NaN,0,319


In [29]:
routes[routes['Equipment'].isna()]

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
2963,7S,\N,RSH,7098,ANI,5967,NaN,0,NaN
2964,7S,\N,SHX,7090,ANI,5967,NaN,0,NaN
3571,9E,3976,ATL,3682,MSP,3858,NaN,0,NaN
3583,9E,3976,GFK,3442,TVF,7018,NaN,0,NaN
3593,9E,3976,MSP,3858,ATL,3682,NaN,0,NaN
3602,9E,3976,TVF,7018,GFK,3442,NaN,0,NaN
4722,AA,24,AUS,3673,MSY,3861,NaN,0,NaN
5909,AA,24,LIR,1881,SJO,1885,NaN,0,NaN
6147,AA,24,MSY,3861,AUS,3673,NaN,0,NaN
14417,BA,1355,ATL,3682,DFW,3670,NaN,0,NaN


In [30]:
routes[routes['Source airport'] == routes['Destination airport']]

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
33276,IL,10121,PKN,3910,PKN,3910,NaN,0,AT7


**Это либо это локальный рейс, который в реальности выполняется как "рейс вокруг" или технический перелёт (но это редкость)**

## 3 задание

В таблицу 1 добавить колонку с названием континента (континенты: Европа, Азия, Африка, Сев. Америка, Юж. Америка, Австралия или Океания, Антарктида).



In [32]:
unique_countries = pd.DataFrame()
unique_countries['Country'] = airports['Country'].unique()
print(unique_countries)

                     Country
0           Papua New Guinea
1                  Greenland
2                    Iceland
3                     Canada
4                    Algeria
..                       ...
232                     Niue
233  Cocos (Keeling) Islands
234                  Myanmar
235                 Svalbard
236              Wake Island

[237 rows x 1 columns]


In [49]:
import pycountry_convert as pc

def country_to_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        return pc.convert_continent_code_to_continent_name(continent_code)
    except:
        return "Unknown"

unique_countries['Continent'] = unique_countries['Country'].apply(country_to_continent)

manual_fixes = {
    'Papua New Guinea': 'Oceania',
    'Greenland': 'North America',
    'Iceland': 'Europe',
    'Canada': 'North America',
    'Algeria': 'Africa',
    'Niue': 'Oceania',
    'Cocos (Keeling) Islands': 'Asia',
    'Myanmar': 'Asia',
    'Svalbard': 'Europe',
    'Wake Island': 'Oceania',
    "Cote d'Ivoire": 'Africa',
    'Congo (Brazzaville)': 'Africa',
    'Congo (Kinshasa)': 'Africa',
    'Saint Helena': 'Africa',
    'Reunion': 'Africa',
    'Antarctica': 'Antarctica',
    'West Bank': 'Asia',
    'Midway Islands': 'Oceania',
    'Virgin Islands': 'North America',
    'Netherlands Antilles': 'South America',
    'Burma': 'Asia',
    'East Timor': 'Asia',
    'Johnston Atoll': 'Oceania',
    'Western Sahara': 'Africa'
}

unique_countries['Continent'] = unique_countries.apply(
    lambda row: manual_fixes.get(row['Country'], row['Continent']),
    axis=1
)

print(unique_countries)


                     Country      Continent
0           Papua New Guinea        Oceania
1                  Greenland  North America
2                    Iceland         Europe
3                     Canada  North America
4                    Algeria         Africa
..                       ...            ...
232                     Niue        Oceania
233  Cocos (Keeling) Islands           Asia
234                  Myanmar           Asia
235                 Svalbard         Europe
236              Wake Island        Oceania

[237 rows x 2 columns]


In [51]:
airports = airports.drop(columns=['Continent'])

airports = airports.merge(unique_countries, on='Country', how='left')

airports.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database timezone,Type,Source,Continent
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,Oceania
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports,Oceania
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports,Oceania
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports,Oceania
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports,Oceania


In [55]:
airports[airports["Country"] == "Svalbard"]

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database timezone,Type,Source,Continent
5592,7451,Svea Airport,Sveagruva,Svalbard,\N,ENSA,77.896944,16.7250,32,1,U,Arctic/Longyearbyen,airport,OurAirports,Europe
5593,7452,Ny-Ålesund Airport (Hamnerabben),Ny-Alesund,Svalbard,\N,ENAS,78.927498,11.8743,50,1,U,Arctic/Longyearbyen,airport,OurAirports,Europe
